<a href="https://colab.research.google.com/github/gigaanderson/ETL/blob/main/Teste_coleta_instagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código Python para acessar o instagram, por meio de uma person válida, e fazer mineração de posts usando scraping via Selenium

In [3]:
import time
import json
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth
from fake_useragent import UserAgent

In [4]:
# Configuração do User-Agent aleatório
ua = UserAgent()
user_agent = ua.random

In [ ]:
# Configuração do WebDriver com Stealth
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument(f"user-agent={user_agent}")

driver = webdriver.Chrome(options=options)

In [ ]:
# Aplicar Stealth para camuflar automação
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)

In [ ]:
# Credenciais do Instagram
USERNAME = "giga.anderson@gmail.com"
PASSWORD = "giga@i2024"

In [ ]:
#Funções

def random_delay(min_sec=2, max_sec=5):
    """Função para adicionar delays aleatórios"""
    time.sleep(random.uniform(min_sec, max_sec))

def login_instagram():
    """Faz login no Instagram"""
    driver.get("https://www.instagram.com/accounts/login/")
    random_delay(5, 8)

    username_input = driver.find_element(By.NAME, "username")
    password_input = driver.find_element(By.NAME, "password")

    username_input.send_keys(USERNAME)
    random_delay(2, 4)
    password_input.send_keys(PASSWORD)
    random_delay(2, 4)
    password_input.send_keys(Keys.RETURN)

    random_delay(8, 12)  # Tempo extra para evitar bloqueios

def buscar_postagens(palavra_chave):
    """Busca postagens que contêm a palavra-chave"""
    search_url = f"https://www.instagram.com/explore/tags/{palavra_chave}/"
    driver.get(search_url)
    random_delay(5, 8)

    # Scroll para carregar mais postagens
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 1000);")
        random_delay(4, 7)

    # Coletar links das postagens
    post_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    links = [post.get_attribute("href") for post in post_links]

    print(f"\n🔍 {len(links)} postagens encontradas com #{palavra_chave}")

    return links

def coletar_detalhes(links):
    """Coleta e exibe detalhes das postagens"""
    resultados = []

    for link in links[:5]:  # Limita a coleta para evitar bloqueios
        driver.get(link)
        random_delay(4, 8)

        try:
            caption = driver.find_element(By.XPATH, "//span").text
        except:
            caption = "Sem legenda"

        try:
            author = driver.find_element(By.XPATH, "//a[contains(@href, '/')]").text
        except:
            author = "Desconhecido"

        post_info = {
            "Autor": author,
            "Legenda": caption[:200] + "..." if len(caption) > 200 else caption,
            "Link": link
        }
        resultados.append(post_info)

        print("\n📌 Postagem encontrada:")
        print(f"🔗 Link: {post_info['Link']}")
        print(f"👤 Autor: {post_info['Autor']}")
        print(f"📝 Legenda: {post_info['Legenda']}")

    return resultados

def salvar_json(dados, nome_arquivo="resultados.json"):
    """Salva os dados coletados em um arquivo JSON"""
    with open(nome_arquivo, "w", encoding="utf-8") as file:
        json.dump(dados, file, ensure_ascii=False, indent=4)
    print(f"\n✅ Dados salvos em {nome_arquivo}")

In [ ]:
    #Executando o Processo e Salvando no JSON final.
    login_instagram()
    links = buscar_postagens("Safelabs")
    resultados = coletar_detalhes(links)
    salvar_json(resultados)
    driver.quit()